In [78]:
# Setup 
from flask import Flask, render_template, request
import os
import pygraphviz as pgv
import networkx as nx
import requests
import json

PATH = '/home/teijehidde/Documents/Git Blog and Coding/Project one (wikipedia SNA)/Code/'
DATA_FILE = 'networkdata.json'
WIKI_URL = "https://en.wikipedia.org"
API_ENDPOINT = WIKI_URL + "/w/api.php"
LIMIT_LINKS_PER_NODE = 500
LIMIT_API_REQUESTS = 100

In [168]:
# Function A: load data node_title from data file. 
def loadNode(node_title):

    try:
        with open(PATH + DATA_FILE) as json_file:
            network_data = json.load(json_file)
            return network_data[node_title]
    except IOError:
        print("Error: Could not find " + DATA_FILE + ". Please check if file is present in directory, or change DATA_FILE value.")
    else: 
        print("The file " + DATA_FILE + " found, succesfully loaded.")

In [169]:
# Function B: download data node_title from Wikimedia API
def downloadNode(node_title):
    
    # setup 
    edges = []

    # requesting data via wikimedia API.  
    S = requests.Session()
    PARAMS = {
        "action": "query",
        "format": "json",
        "titles": node_title,
        "prop": "links",
        # "plcontinue": ,
        "plnamespace": 0, # only load wikipedia main/articles. 
        "pllimit": 'max' # can go up to 500. Go for max? 
    }
    response = S.get(url=API_ENDPOINT, params=PARAMS)

    # Transforming response to network data format + error handling.  
    data_wiki = response.json()
    node = next(iter(data_wiki['query']['pages']))
    
    try: 
        for x in data_wiki['query']['pages'][node]['links']:
            edges.append(x['title'])
            
        node_data = {'node_ID': node, 'ego': 0, 'date_time': 'TODO', 'edges': edges, 'revisions': 'TODO'}
        network_data[node_title] = node_data
    except:
        print("Error: an exception occured while downloading " + node_title + ".")
    else: 
        print("Links data on page " + node_title + " successfully downloaded.")
        return network_data[node_title]
        with open(PATH + DATA_FILE, 'w') as outfile:
            json.dump(network_data, outfile)
            print("Data succesfully downloaded and saved.")

    # sleep(0.5)

In [170]:
# initiate class nodes. 
class Node:
    def __init__(self,node_title):
        self.title = node_title
        try: 
            self.nodes = loadNode(node_title)['edges']
        except: 
            self.nodes = []
            print("Data on " + node_title + " not available in "  + DATA_FILE + ". Node returned empty")
        
    def getEdges(self):
        try: 
            x = {}
            for item in self.nodes: 
                x[item] = None
            return x
        except TypeError:
            return "Error: " + self.title + " is an empty node."

    def getUpdate(self): 
        try:
            self.nodes = downloadNode(self.title)['edges']
        except:
            self.nodes = None
            print("This title does not seem to exist on " + WIKI_URL + ".")


In [171]:
class Network(Node):
    def __init__(self, node_title):
        Node.__init__(self, node_title)

    def getNodesNetwork(self):
        nodes_network = set() 
        for item in self.nodes: 
            x = Node(item)
            nodes_network = nodes_network | set(x.nodes)
        return nodes_network

    def getEdgesNetwork(self):
        edges_network = {} 
        for item in self.nodes: 
            x = Node(item)
            edges_network[item] = x.getEdges()
        return edges_network


In [172]:
terrorism_network = Network("Terrorism")

In [173]:
test = terrorism_network.getEdgesNetwork() 

Data on Allied of World War II not available in networkdata.json. Node returned empty
Data on Dawabsheh not available in networkdata.json. Node returned empty
Data on International Journal of Disaster Medicine not available in networkdata.json. Node returned empty


In [ ]:
type(test)

In [180]:
G = pgv.AGraph(test)
# s = G.string()
G.write(PATH + "test.dot") 
G.layout(prog="dot") 
G.draw(PATH + "test.png")

500